# 训练数据生成
进行内部数据转换，生成模型微调所需训练和测试数据

## 功能性函数封装

In [1]:
import json
import os
import numpy as np
import jieba.posseg as pseg
import re

# 构造标准列名映射
schema_mapping = {
    "国别": "国家",
    "主体名称": "相关主体",
    "主体类型": "类型",
    "主体数量": "数量",
    "装备名称": "相关主体",
    "装备类型": "类型",
    "装备数量": "数量",
    "装备": "相关主体",
    # "数量": "数量",
    "开始时间": "时间",
    "任务类型": "任务"
}

# 构造头实体备选列
header_schema = ["任务", "行动", "事件"]

# 构造低质量及不需要的schema排除列
remove_list = ["国家", "人物", "备注", "国别", "任务", "行动", "事件", "类型", "数量", "人数"]

retain_list = ["军种","单位","任务状态"]

# 判断表格内容是否有效（可保留）
def retain_entity(value):
    if value == "" or ("未提及" in value) or ("未明确" in value) or ("未提供" in value) or ("未知" in value):
        return False
    else:
        return True

# 判断表格内容是不是包含“，、”的长句子
def not_sentence(value):
    if "，" in value or "、" in value:
        return False
    else:
        return True

# 获取动词最多的分句
def get_verb_head(text):
    #  使用正则表达式同时根据逗号和顿号拆分句子
    # 正则表达式中的 | 表示逻辑或，[^ ] 表示匹配括号内的任意字符
    sentences = re.split(r'，|、', text)
    # 初始化变量，用于记录动词最多的分句
    max_verb_count = 0
    verb_sent = ""
    # 对每个句子进行处理
    for sent in sentences:
        # 使用jieba进行分词和词性标注
        words = pseg.cut(sent)
        # 统计当前句子的动词数量
        verb_count = sum(1 for word, flag in words if (flag == 'v' or flag == 'vn')) 
        # 更新动词最多的分句
        if verb_count > max_verb_count:
            max_verb_count = verb_count
            verb_sent = sent     
    # 返回动词含量最多的分句
    return verb_sent

# 判断是不是相同的关系
def are_objects_equal(obj1, obj2):
    if sorted(obj1.items()) == sorted(obj2.items()):
        return True

# 去除低质量的数量关系（加和那种）
def remove_add_relation(relation_array):
    # 结果列表
    unique_list = []
    # 用于记录已经检查过的head和relation的组合
    checked_combinations = set()
    # 遍历原始列表
    for item in relation_array:
        # 检查是否满足剔除条件：head和relation相同，relation为"数量"，tail不同
        combination = (item["head"], item["relation"])
        if item["relation"] == "数量" and combination in checked_combinations:
            # 如果tail不同，则不添加任何对象
            continue
        # 添加对象到结果列表
        unique_list.append(item)
        # 记录这个组合，无论是否满足剔除条件
        checked_combinations.add(combination)
        # 如果relation为"数量"，检查是否有相同head和relation但不同tail的对象
        if item["relation"] == "数量":
            for other_item in relation_array:
                if other_item != item and (other_item["head"], other_item["relation"]) == combination and other_item["tail"] != item["tail"]:
                    # 找到相同head和relation但不同tail的对象，剔除
                    unique_list.pop()
                    break   
    return unique_list

# 在去除加和劣质数据基础上，去除重复关系得到最终的关系数组
def get_final_relations(relation_array):
    temp_list = remove_add_relation(relation_array)
    seen = []
    unique_list = []
    for item in temp_list:
        if not any(are_objects_equal(item, seen_item) for seen_item in seen):
            seen.append(item)
            unique_list.append(item)
    return unique_list

## 标注数据文件地址

In [12]:
# 从 JSON 文件中读取数据
input_file_path = 'data2text-test-remove-rare.json'

train_output_file_path = 'remove-rare-train-val.json'

test_output_file_path = 'remove-rare-test.json'

update_sample_path = 'RE-update-all-v4.json'

rare_input_path = 'rare_data.json'

rare_sample_path = 'rare-all.json'

remove_rare_path = 'remove-rare-all.json'

## 统计不同的表头

表头类型统计
表头1：["时间","人物","国家","装备名称","装备类型","装备数量","组织机构","地点","任务","行动","事件"] 1161
- 处理规则：
  - [ ] 装备相关schema改为“主体”
  - [ ] 以“行动”内容作为事件触发词（“行动”有些内容也是长句子的表述，或者没有意义）
    - 规则1：“行动”表格内容可能没有意义，此时转为判断“任务”表格内容是否有意义
    - 规则2：“行动”有些内容也是长句子的表述，判断字符串内如果有逗号，则关注“任务”表格内容
    - 规则3：接续规则2，若“行动”是长句表述，但“任务”为空，则把“行动”根据逗号分割，取出其中一串作为触发词

表头2：["时间","人物","国家","主体名称","主体类型","主体数量","组织机构","地点","行动","事件"] 967
- 处理规则：
  - [ ] 以“行动”内容作为事件触发词（“行动”有些内容也是长句子的表述，或者没有意义）
    - 规则1：“行动”表格内容可能没有意义，此时舍弃该条数据
    - 规则2：“行动”有些内容也是长句子的表述，判断字符串内如果有逗号，则把“行动”根据逗号分割，取出其中一串作为触发词

表头3：["国别", "军种", "单位", "装备", "人数", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"] 7
- 处理规则：
  - [ ] 以“行动”内容作为事件触发词（“行动”有些内容也是长句子的表述，或者没有意义）
    - 规则1：“行动”表格内容可能没有意义，此时舍弃该条数据
    - 规则2：“行动”有些内容也是长句子的表述，判断字符串内如果有逗号，则把“行动”根据逗号分割，取出其中一串作为触发词
  - [ ] “装备”表格内容需要按照分号、逗号进行拆分，分割成多个独立的记录
  - [ ] “指挥员”表格内容暂未发现
表头4：['国别', '军种', '装备数量', '装备', '地点', '行动', '开始时间', '任务状态'] 2

表头5：['国别', '军种', '单位', '装备', '装备数量', '指挥员', '地点', '行动', '开始时间', '任务状态', '任务类型', '备注'] 4

表头6：['军种', '数量', '装备', '地点', '行动', '任务状态'] 5

表头7：['国别', '军种', '单位', '装备', '数量', '指挥员', '地点', '行动', '开始时间', '任务状态', '任务类型', '备注'] 1

表头8：['国别', '军种', '数量', '装备', '地点', '行动', '开始时间', '任务状态'] 1


### 需要对单元格内容做分隔处理的数据
表头3：["国别", "军种", "单位", "装备", "人数", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"]

{   "text": "7月7日，空军第九运输机师的运-20运输机（代号KJ-2000）8架，前往青藏高原某机场进行高原起降训练。此次训练旨在提高运输机在高原复杂环境下的操作能力，保障部队快速机动。7月8日，海军第六驱逐舰支队的大连舰（052D驱逐舰，舷号110）和综保支队的微山湖舰（综合补给舰，舷号887）共328人，前往夏威夷参加环太平洋-2024演习。演习期间，大连舰进行了反舰导弹发射训练。", 
    "event": [{"event_trigger": "环太平洋-2024演习", "event_type": "行动", "arguments": [{"argument": "1艘", "role": "主体数量"}, {"argument": "052D驱逐舰", "role": "主体名称"}]}]
}

In [22]:

headers = []

# headers.append(["时间","人物","国家","装备名称","装备类型","装备数量","组织机构","地点","任务","行动","事件"])
count=0
count_all=0
for data in input_data:
    count_all+=1
    # 1161
    array1 = ["时间","人物","国家","装备名称","装备类型","装备数量","组织机构","地点","任务","行动","事件"]
    a1 = np.array(array1)
    # 967
    array2 = ["时间","人物","国家","主体名称","主体类型","主体数量","组织机构","地点","行动","事件"]
    a2 = np.array(array2)
    # 7
    array3 = ["国别", "军种", "单位", "装备", "人数", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"]
    a3 = np.array(array3)
    # 2
    array5 = ['国别', '军种', '装备数量', '装备', '地点', '行动', '开始时间', '任务状态']
    a5 = np.array(array5)
    # 4
    array6 = ['国别', '军种', '单位', '装备', '装备数量', '指挥员', '地点', '行动', '开始时间', '任务状态', '任务类型', '备注']
    a6 = np.array(array6)
    # 5
    array7 = ['军种', '数量', '装备', '地点', '行动', '任务状态']
    a7 = np.array(array7)
    # 1
    array8 = ['国别', '军种', '单位', '装备', '数量', '指挥员', '地点', '行动', '开始时间', '任务状态', '任务类型', '备注']
    a8 = np.array(array8)
    # 1
    array9 = ['国别', '军种', '数量', '装备', '地点', '行动', '开始时间', '任务状态']
    a9 = np.array(array9)
    if isinstance(data, list):
        for row in data:
            array4 = row['table']['header']
            a4 = np.array(array4)
            if np.array_equal(a9, a4):
                count+=1
            # if np.array_equal(a1, a4) or np.array_equal(a2, a4) or np.array_equal(a3, a4) or np.array_equal(a5, a4) or np.array_equal(a6, a4) or np.array_equal(a7, a4):
            #     pass
            # else:
            #     headers.append(array4)



                # array1 = array2
                # a1 = np.array(array1)
    # if isinstance(data, list):
    #     for row in data:
    #         array4 = row['table']['header']
    #         a4 = np.array(array4)
    #         if np.array_equal(a1, a4):
    #             pass
    #         else:
    #             headers.append(array4)
    #             array1 = array4.copy()
    #             a1 = np.array(array1)     


# 打印集合内容
# print("不同的 表头 有：", headers)
print("表头8数量：", count)
print("数据总数：", count_all)
# 打印总共有多少种不同的 entity_type
# print("共有", len(entity_types), "种不同的 entity_type")

表头8数量： 1
数据总数： 2138


## 生成训练集和测试集（EE）

In [8]:
import numpy as np

train_val_set = []
test_set = []
all_set = []

header1 = ["时间","人物","国家","装备名称","装备类型","装备数量","组织机构","地点","任务","行动","事件"]
header2 = ["时间","人物","国家","主体名称","主体类型","主体数量","组织机构","地点","行动","事件"]
header3 = ["国别", "军种", "单位", "装备", "数量", "人数", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"]
header4 = ["国别", "军种", "装备数量", "装备", "地点", "行动", "开始时间", "任务状态"]
header5 = ["国别", "军种", "单位", "装备", "装备数量", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"]
header6 = ["军种", "数量", "装备", "地点", "行动", "任务状态"]
header7 = ["国别", "军种", "单位", "装备", "数量", "指挥员", "地点", "行动", "开始时间", "任务状态", "任务类型", "备注"]
header8 = ["国别", "军种", "数量", "装备", "地点", "行动", "开始时间", "任务状态"]

# 遍历输入数据中的每个data项
for excel in input_data:
    for sheet in excel:
        events = []
        relation_type = ""
        update_header = ["时间","人物","国家","主体名称","主体类型","主体数量","组织机构","地点","行动","事件"]
        # 判断当前的表头类型
        if np.array_equal(np.array(sheet['table']['header']),np.array(header1)):
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            update_header = ["时间","人物","国家","主体名称","主体类型","主体数量","组织机构","地点","任务","行动","事件"]
            # 获取可能的中心词 行动、任务 的下标
            head_index_action = sheet['table']['header'].index("行动")
            head_index_task = sheet['table']['header'].index("任务")

            # 遍历data中的每一行
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 、 任务 的表格内容
                action = row_value[head_index_action]
                task = row_value[head_index_task]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         # 行动 有意义但是长句子，则先看一下 任务 单元格内容是否有效
                         if retain_entity(task) and not_sentence(task):
                             # 任务 单元格有效且不是长句子，则把任务内容作为trigger
                             event_trigger = task
                         else:
                             # 任务 单元格无效或是长句子则回头拆分 行动 单元格内容,近似选择一段字符串
                             items = action.split('，')
                             event_trigger = items[0]
                # 行动单元格无意义，判断 任务 单元格内容
                else:
                    if retain_entity(task):
                        if not_sentence(task):
                             # 任务 单元格有效且不是长句子，则把任务内容作为trigger
                             event_trigger = task
                        else:
                             # 任务 单元格有效但是长句子，分割后任务内容作为trigger
                             items = task.split('，')
                             event_trigger = items[0]
                    else:
                        # 行动和任务单元格都没意义，这条数据舍弃
                        pass
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国家", "人物", "任务", "行动", "事件"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)

        
        if np.array_equal(np.array(sheet['table']['header']),np.array(header2)):
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            # 获取可能的中心词 行动 的下标
            head_index_action = sheet['table']['header'].index("行动")
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 的表格内容
                action = row_value[head_index_action]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         items = action.split('，')
                         event_trigger = items[0]
                else:
                    # 行动单元格都没意义，这条数据舍弃
                    pass
    
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国家", "人物", "任务", "行动", "事件"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)

        
        if np.array_equal(np.array(sheet['table']['header']),np.array(header3)):
            update_header = ["国别", "军种", "单位", "主体名称", "主体数量", "人数", "指挥员", "地点", "行动", "时间", "任务状态", "任务类型", "备注"]
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            # 获取可能的中心词 行动 的下标
            head_index_action = sheet['table']['header'].index("行动")
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 的表格内容
                action = row_value[head_index_action]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         items = action.split('，')
                         event_trigger = items[0]
                else:
                    # 行动单元格都没意义，这条数据舍弃
                    pass
    
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国别", "人物", "任务", "行动", "事件", "备注"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)

        
        if np.array_equal(np.array(sheet['table']['header']),np.array(header4)):
            update_header = ["国别", "军种", "主体数量", "主体名称", "地点", "行动", "时间", "任务状态"]
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            # 获取可能的中心词 行动 的下标
            head_index_action = sheet['table']['header'].index("行动")
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 的表格内容
                action = row_value[head_index_action]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         items = action.split('，')
                         event_trigger = items[0]
                else:
                    # 行动单元格都没意义，这条数据舍弃
                    pass
    
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国别", "人物", "任务", "行动", "事件", "备注"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)


        if np.array_equal(np.array(sheet['table']['header']),np.array(header5)):
            update_header = ["国别", "军种", "单位", "主体名称", "主体数量", "指挥员", "地点", "行动", "时间", "任务状态", "任务类型", "备注"]
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            # 获取可能的中心词 行动 的下标
            head_index_action = sheet['table']['header'].index("行动")
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 的表格内容
                action = row_value[head_index_action]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         items = action.split('，')
                         event_trigger = items[0]
                else:
                    # 行动单元格都没意义，这条数据舍弃
                    pass
    
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国别", "人物", "任务", "行动", "事件", "备注"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)


        if np.array_equal(np.array(sheet['table']['header']),np.array(header6)):
            update_header = ["军种", "主体数量", "主体名称", "地点", "行动", "任务状态"]
            # 统一一下装备和主体的命名，尽量减少实体数量，但整体结构不会变动
            # 获取可能的中心词 行动 的下标
            head_index_action = sheet['table']['header'].index("行动")
            for row, row_value in sheet['table']['data'].items(): 
                arguments = []
                event_trigger = ""
                event_type = "事件"
                # 获取 行动 的表格内容
                action = row_value[head_index_action]
                # 判断 行动 表格内容是否有意义
                if retain_entity(action):
                    # 有意义则进一步判断是不是长句子
                     if not_sentence(action):
                         # 不是长句子，直接把内容作为trigger
                         event_trigger = action
                     else:
                         items = action.split('，')
                         event_trigger = items[0]
                else:
                    # 行动单元格都没意义，这条数据舍弃
                    pass
    
                if event_trigger != "":
                    # 确定触发词有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        header_index = row_value.index(value)
                        entity_type = update_header[header_index]
                        if entity_type not in ["国别", "人物", "任务", "行动", "事件", "备注"]:
                            if retain_entity(value):
                                arguments.append({
                                    "argument": value,
                                    "role": entity_type})
                    events.append({
                        "event_trigger": event_trigger,
                        "event_type": event_type,
                        "arguments": arguments
                    })
                # 创建输出格式
                if len(events):
                    output = {"text": sheet["text"], "event": events}
                    all_set.append(output)




# 打乱数组顺序
np.random.shuffle(all_set)

# 计算分割点
split_index = int(len(all_set) * 0.9)
print(len(all_set))

# 划分训练集和测试集
train_val_set = all_set[:split_index]
test_set = all_set[split_index:]

# 获取当前工作目录
current_path = os.getcwd()
output_file_path = os.path.join(current_path, '../iepile-ee-train-val.json')
test_file_path = os.path.join(current_path, '../iepile-ee-test.json')

# 打开输出路径
with open(output_file_path, 'w', encoding='utf-8') as file1:
    json.dump(train_val_set, file1, ensure_ascii=False, indent=1)
with open(test_file_path, 'w', encoding='utf-8') as file2:
    json.dump(test_set, file2, ensure_ascii=False, indent=1)

3326


## 生成测试数据函数封装-RE

In [16]:
def data_premanage(input_path, standard_schema_map, header_schema, remove_list, train_output_path, test_output_path):
# def data_premanage(input_path, standard_schema_map, header_schema, remove_list, train_output_path):

    with open(input_path, 'r', encoding='utf-8') as file:
        input_data = json.load(file)

    train_val_set = []
    test_set = []
    all_set = []

    # 遍历输入数据中的每个data项
    for excel in input_data:
        for sheet in excel:
            # 获取原始表头，并根据标准列映射进行替换，更新得到标准表头
            header = sheet['table']['header']
            for i in range(len(header)):
                for key, value in standard_schema_map.items():
                    if header[i] == key:
                        header[i] = value
            relations = []
            # 遍历data中的每一行
            for row, row_value in sheet['table']['data'].items(): 
                head_type = "",
                head_value = "",
                # 获取可能的中心词的下标( 任务 > 行动 > 事件)
                for i in range(len(header_schema)):
                # 从任务开始，如果数据表头中有任务，就先取出任务内容进行判断，后面不行再用i+1
                    if header_schema[i] in header:
                    # 第一个能对应上的候选头实体，拿到其 类型type 和 在当前表头中的数组下标index
                        head_index = header.index(header_schema[i])
                        head_type = header_schema[i]
                        # 判断 候选头实体的初始值value 是否有意义
                        if retain_entity(head_value):
                        # 有意义，则用刚才找到的index对应找到 候选头实体的初始值value
                            head_value = row_value[head_index]
                            # 进一步判断是不是长句子
                            if not_sentence(head_value):
                                # 不是长句子，不用再做其他处理，value可直接作为头实体
                                pass
                            else:
                                # 候选头实体的初始值value 有意义但是长句子，则分句后选择动词最多的字段作为头实体
                                head_value = get_verb_head(head_value)
                        else:
                            pass
                        break
                    else:
                        continue
                
                # 开始构造数据
                if retain_entity(head_value):
                     # 确定头实体有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        tail_type_index = row_value.index(value)
                        tail_type = header[tail_type_index]
                        if tail_type not in remove_list:
                            if retain_entity(value):
                                if tail_type == "相关主体":
                                    if "，" in value or "；" in value:
                                        records = re.split(r'；', value)
                                        for record in records:
                                            single_name_value = re.split(r'，', record)[0]
                                            single_num_value = re.split(r'，', record)[1]
                                            relations.append({
                                                "head": head_value,
                                                "relation": tail_type,
                                                "tail": single_name_value})
                                            # relations.append({
                                            #     "head": single_name_value,
                                            #     "relation": "数量",
                                            #     "tail": single_num_value})
                                    else:
                                        relations.append({
                                                "head": head_value,
                                                "relation": tail_type,
                                                "tail": value})
                                elif tail_type == "时间":
                                    if value in sheet["text"]:
                                        relations.append({
                                            "head": head_value,
                                            "relation": tail_type,
                                            "tail": value})
                                else:    
                                    relations.append({
                                        "head": head_value,
                                        "relation": tail_type,
                                        "tail": value})
                
                # 对于 主体名称-主体数量、主体名称-主体类型 两个关系，单独进行构造
                # 先拿到 主体名称 表格内容
                subject_index = header.index("相关主体")
                subject_value = row_value[subject_index]
                # 如果 主体名称 有效，进一步考虑构造
                if retain_entity(subject_value):
                    # 依次获取两个尾实体表格内容
                    for subject_tail in ["类型", "数量"]:
                        if subject_tail in header:
                            subject_tail_index = header.index(subject_tail)
                            subject_tail_value = row_value[subject_tail_index]
                            # 表格内容有效则构建一条关系
                            if retain_entity(subject_tail_value):
                                if subject_tail == "类型":
                                    relations.append({
                                            "head": subject_value,
                                            "relation": subject_tail,
                                            "tail": subject_tail_value})
                                # if subject_tail == "数量" and subject_tail_value in sheet["text"]:
                                #     relations.append({
                                #             "head": subject_value,
                                #             "relation": subject_tail,
                                #             "tail": subject_tail_value})

            final_relations = get_final_relations(relations)
            # 创建输出格式
            if len(relations):
                output = {"text": sheet["text"], "relation": final_relations}
                all_set.append(output)

    # 打乱数组顺序
    np.random.shuffle(all_set)

    # 计算分割点
    split_index = int(len(all_set) * 0.9)
    print(len(all_set))

    # 划分训练集和测试集
    train_val_set = all_set[:split_index]
    test_set = all_set[split_index:]

    # 获取当前工作目录
    current_path = os.getcwd()
    output_file_path = os.path.join(current_path, train_output_path)
    test_file_path = os.path.join(current_path, test_output_path)

    # 打开输出路径
    with open(output_file_path, 'w', encoding='utf-8') as file1:
        json.dump(train_val_set, file1, ensure_ascii=False, indent=1)
    with open(test_file_path, 'w', encoding='utf-8') as file2:
        json.dump(test_set, file2, ensure_ascii=False, indent=1)



    print(len(all_set))
    # # 获取当前工作目录
    # current_path = os.getcwd()
    # output_file_path = os.path.join(current_path, train_output_path)
    # # test_file_path = os.path.join(current_path, test_output_path)

    # # 打开输出路径
    # with open(output_file_path, 'w', encoding='utf-8') as file1:
    #     json.dump(all_set, file1, ensure_ascii=False, indent=1)
    # # with open(test_file_path, 'w', encoding='utf-8') as file2:
    # #     json.dump(test_set, file2, ensure_ascii=False, indent=1)

## rare_data处理

In [6]:
# def data_premanage(input_path, standard_schema_map, header_schema, remove_list, train_output_path, test_output_path):
def data_premanage(input_path, standard_schema_map, header_schema, retain_list, train_output_path):

    with open(input_path, 'r', encoding='utf-8') as file:
        input_data = json.load(file)

    train_val_set = []
    test_set = []
    all_set = []

    # 遍历输入数据中的每个data项
    for excel in input_data:
        for sheet in excel:
            # 获取原始表头，并根据标准列映射进行替换，更新得到标准表头
            header = sheet['table']['header']
            for i in range(len(header)):
                for key, value in standard_schema_map.items():
                    if header[i] == key:
                        header[i] = value
            relations = []
            # 遍历data中的每一行
            for row, row_value in sheet['table']['data'].items(): 
                head_type = "",
                head_value = "",
                # 获取可能的中心词的下标( 任务 > 行动 > 事件)
                for i in range(len(header_schema)):
                # 从任务开始，如果数据表头中有任务，就先取出任务内容进行判断，后面不行再用i+1
                    if header_schema[i] in header:
                    # 第一个能对应上的候选头实体，拿到其 类型type 和 在当前表头中的数组下标index
                        head_index = header.index(header_schema[i])
                        head_type = header_schema[i]
                        # 判断 候选头实体的初始值value 是否有意义
                        if retain_entity(head_value):
                        # 有意义，则用刚才找到的index对应找到 候选头实体的初始值value
                            head_value = row_value[head_index]
                            # 进一步判断是不是长句子
                            if not_sentence(head_value):
                                # 不是长句子，不用再做其他处理，value可直接作为头实体
                                pass
                            else:
                                # 候选头实体的初始值value 有意义但是长句子，则分句后选择动词最多的字段作为头实体
                                head_value = get_verb_head(head_value)
                        else:
                            pass
                        break
                    else:
                        continue
                
                # 开始构造数据
                if retain_entity(head_value):
                     # 确定头实体有效才开始构建数据
                    for value in row_value:
                        # 对应schema和单元格内容value
                        tail_type_index = row_value.index(value)
                        tail_type = header[tail_type_index]
                        if tail_type in retain_list:
                            if retain_entity(value):  
                                relations.append({
                                    "head": head_value,
                                    "relation": tail_type,
                                    "tail": value})
            
            final_relations = get_final_relations(relations)
            # 创建输出格式
            if len(relations):
                output = {"text": sheet["text"], "relation": final_relations}
                all_set.append(output)

    # # 打乱数组顺序
    # np.random.shuffle(all_set)

    # # 计算分割点
    # split_index = int(len(all_set) * 0.9)
    # print(len(all_set))

    # # 划分训练集和测试集
    # train_val_set = all_set[:split_index]
    # test_set = all_set[split_index:]

    # # 获取当前工作目录
    # current_path = os.getcwd()
    # output_file_path = os.path.join(current_path, train_output_path)
    # test_file_path = os.path.join(current_path, test_output_path)

    # # 打开输出路径
    # with open(output_file_path, 'w', encoding='utf-8') as file1:
    #     json.dump(train_val_set, file1, ensure_ascii=False, indent=1)
    # with open(test_file_path, 'w', encoding='utf-8') as file2:
    #     json.dump(test_set, file2, ensure_ascii=False, indent=1)



    print(len(all_set))
    # 获取当前工作目录
    current_path = os.getcwd()
    output_file_path = os.path.join(current_path, train_output_path)
    # test_file_path = os.path.join(current_path, test_output_path)

    # 打开输出路径
    with open(output_file_path, 'w', encoding='utf-8') as file1:
        json.dump(all_set, file1, ensure_ascii=False, indent=1)
    # with open(test_file_path, 'w', encoding='utf-8') as file2:
    #     json.dump(test_set, file2, ensure_ascii=False, indent=1)

In [25]:
input_path = 'rare-test.json'
output_file_path = 'enhance-rare-test-all.json'

with open(input_path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

np.random.shuffle(input_data)
# copied_data = []

# for item in input_data:
#     for _ in range(56):
#         copied_data.append(item)

# print(len(copied_data))

with open(output_file_path, 'w', encoding='utf-8') as file1:
    json.dump(input_data, file1, ensure_ascii=False, indent=1)

## 运行测试

In [17]:
data_premanage(input_file_path, schema_mapping, header_schema, remove_list, train_output_file_path, test_output_file_path)

# data_premanage(input_file_path, schema_mapping, header_schema, remove_list, remove_rare_path)

#data_premanage(rare_input_path, schema_mapping, header_schema, retain_list, rare_sample_path)

1763
1763


## 转变为jsonl格式并直接重命名为json

In [27]:
import json
import jsonlines
import os


def rename_file(old_name, new_name):
    # 确保旧文件存在
    if os.path.exists(old_name):
        # 重命名文件
        os.rename(old_name, new_name)
        print(f"File renamed from {old_name} to {new_name}")
    else:
        print(f"File {old_name} does not exist.")


train_json_file = "enhance-rare-train-val-all.json"
train_jsonl_file = "enhance-rare-train-val-all.jsonl"
test_json_file = "enhance-rare-test-all.json"
test_jsonl_file = "enhance-rare-test-all.jsonl"

with open(train_json_file, "r", encoding='utf-8') as file:
    with jsonlines.open(train_jsonl_file, 'w') as writer:
        input_data = json.load(file)
        for data in input_data:
            writer.write(data)

with open(test_json_file, "r", encoding='utf-8') as file:
    with jsonlines.open(test_jsonl_file, 'w') as writer:
        input_data = json.load(file)
        for data in input_data:
            writer.write(data)

rename_file("enhance-rare-train-val-all.jsonl", "enhance-rare-train-val-json.json")
rename_file("enhance-rare-test-all.jsonl", "enhance-rare-test-json.json")


File renamed from enhance-rare-train-val-all.jsonl to enhance-rare-train-val-json.json
File renamed from enhance-rare-test-all.jsonl to enhance-rare-test-json.json


## 生成测试集（按照单个schema）
1. 按照单个schema生成测试集，目前其他测试方法优先级更高，此处代码保留但不作运行

In [7]:
input_file_path_test = 'NERsample_test.json'

with open(input_file_path_test, 'r', encoding='utf-8') as file:
    input_data_test = json.load(file)

# 获取当前工作目录
current_path = os.getcwd()
output_file_path = os.path.join(current_path, 'schema_NERsample_test.json')

output_data_test = []
# 遍历输入数据中的每个data项
for entity_type in ["时间", "人物", "国家", "装备名称", "装备类型", "装备数量", "组织机构", "地点", "任务", "行动", "事件"]:
    output_file_path = os.path.join(current_path, entity_type + 'NERsample_test.json')
    for data in input_data_test:
        entities = []
        for item in data['entity']:
            if item["entity_type"] == entity_type:
                entities.append(item)
                output = {"text": data["text"], "entity": entities}
                output_data_test.append(output)
    # 打开输出路径
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data_test, file, ensure_ascii=False,indent=1)


In [29]:
input_file_path_test = 'NERsample_test.json'

with open(input_file_path_test, 'r', encoding='utf-8') as file:
    input_data_test = json.load(file)

# 获取当前工作目录
current_path = os.getcwd()

output_data_test = []
# 遍历输入数据中的每个data项

output_file_path = os.path.join(current_path,  '装备数量NERsample_test.json')
for data in input_data_test:
    for item in data['entity']:
        if item["entity_type"] == "装备数量":
            entities = []
            entities.append(item)
            output = {"text": data["text"], "entity": entities}
            output_data_test.append(output)
# 打开输出路径
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(output_data_test, file, ensure_ascii=False,indent=1)

In [10]:
def find_all_spans(source_text, substring):
    length = len(substring)
    spans = []
    for i in range(len(source_text) - length + 1):
        if source_text[i:i+length] == substring:
            spans.append((i, i + length))
    return spans


source_text = "这是一个测试文本，这包含一些用于测试的子字符串。"
substring = "字符串"

spans = find_all_spans(source_text, substring)
print(spans)
# span = find_span(source_text, substring)
# print(span)  # 输出子字符串的位置span

[(20, 23)]


### 将训练数据转换为模型微调使用的数据格式
运行格式转换脚本进行格式转换，其中指令含义如下：
- --src_path data2text/iepile-ner-train-val-json.json
   - 训练数据的路径
- --tgt_path data2text/iepile-ner-train-val-transformed.json
   - 数据转换之后文件输出的路径
- --language zh
   - 选择zh语言
- --task NER
   - 选择实体抽取任务
- --split_num 4
   - 定义单个指令中可包含的最大schema数目
- --random_sort
   - 是否对指令中的schema随机排序，默认为false
- --split train
   - 指定数据集类型为train

In [28]:
!python ../../ie2instruction/convert_func.py \
    --src_path improve_data/enhance-rare-train-val-json.json \
    --tgt_path improve_data/enhance-rare-train-val-transformed.json \
    --schema_path improve_data/schema.json \
    --language zh \
    --task RE \
    --split_num 6 \
    --random_sort \
    --split train

Traceback (most recent call last):
  File "/root/data2text-oneke/data2text_RE/improve_data/../../ie2instruction/convert_func.py", line 170, in <module>
    process(options)
  File "/root/data2text-oneke/data2text_RE/improve_data/../../ie2instruction/convert_func.py", line 107, in process
    options.source = options.src_path.split('/')[-2]  # 用源路径的最后一个文件夹名作为source
IndexError: list index out of range


### 拆分训练集和验证集

In [8]:
import json
import random


# 读取JSONL文件
def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    return data


# 划分数据集
def split_data(data, train_ratio=0.8):
    random.shuffle(data)  # 随机打乱数据
    train_size = int(len(data) * train_ratio)
    train_data = data[:train_size]
    val_data = data[train_size:]
    return train_data, val_data


# 写入JSONL文件
def write_jsonl(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in data:
            file.write(json.dumps(entry, ensure_ascii=False) + '\n')


# 主函数
def main(input_file, train_file, val_file):
    data = read_jsonl(input_file)
    train_data, val_data = split_data(data)
    write_jsonl(train_data, train_file)
    write_jsonl(val_data, val_file)
    print(f"训练集和验证集已生成：\n训练集大小：{len(train_data)}\n验证集大小：{len(val_data)}")


# 使用示例
input_file = 'newdata/iepile-re-train-val-transformed.json'  # 输入的JSONL文件
train_file = 'newdata/iepile-re-train-transformed.json'  # 训练集文件
val_file = 'newdata/iepile-re-val-transformed.json'      # 验证集文件

main(input_file, train_file, val_file)


训练集和验证集已生成：
训练集大小：3491
验证集大小：873


### 将测试数据转换为测试使用的数据格式
运行格式转换脚本进行格式转换，其中指令含义如下：
- --src_path data2text/iepile-ner-test-json.json
   - 训练数据的路径
- --tgt_path data2text/iepile-ner-test-transformed.json
   - 数据转换之后文件输出的路径
- --language zh
   - 选择zh语言
- --task NER
   - 选择实体抽取任务
- --split_num 4
   - 定义单个指令中可包含的最大schema数目
- --random_sort
   - 是否对指令中的schema随机排序，默认为false
- --split test
   - 指定数据集类型为train

In [11]:
!python ../../ie2instruction/convert_func.py \
    --src_path newdata/test_demo.json \
    --tgt_path newdata/test_demo-transformed.json \
    --schema_path newdata/schema_test.json \
    --language zh \
    --task RE \
    --split_num 6 \
    --split test

## 将测试输出每6条合并
测试集的输出因为split_num = 4，故每条文本被拆分为6条数据输出，我们需要将其合并

In [5]:
import json


def merge_dicts(dict_list):
    """合并多个字典，将相同键的值合并为一个列表"""
    merged_dict = {}
    for d in dict_list:
        for key, value in d.items():
            if key in merged_dict:
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = value
    return merged_dict


def process_jsonl_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    result = []
    temp_dict_list = []

    for i, line in enumerate(lines):
        data = json.loads(line)
        output_data = json.loads(data['output'])
        temp_dict_list.append(output_data)

        # 每6条数据合并一次
        if (i + 1) % 6 == 0 or i == len(lines) - 1:
            merged_output = merge_dicts(temp_dict_list)
            result.append({"result": merged_output})
            temp_dict_list = []

    # 将结果写入JSON文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)


# 使用示例
input_file = 'results/baichuan7B-ft-test-output.json'
output_file = 'results/baichuan7B-ft-test-output-merged.json'

process_jsonl_file(input_file, output_file)


## 单条分析

### 数据

In [ ]:
{
            "table": {
                "header": [
                    "时间",
                    "人物",
                    "国家",
                    "装备名称",
                    "装备类型",
                    "装备数量",
                    "组织机构",
                    "地点",
                    "任务",
                    "行动",
                    "事件"
                ],
                "data": {
                    "2": [
                        "未提供具体时间",
                        "谢尔盖・里亚布科夫",
                        "俄罗斯",
                        "新型高超声速武器",
                        "高超声速武器",
                        "未提供数量",
                        "俄罗斯外交部",
                        "未提供具体地点",
                        "与美国展开对话",
                        "就新型高超声速武器展开对话",
                        "俄罗斯愿意与美国就高超声速武器展开对话"
                    ]
                }
            },
            "origin": {},
            "text": "俄罗斯外交部副部长谢尔盖・里亚布科夫表示，俄罗斯愿意与美国就俄罗斯研发的新型高超声速武器展开对话，前提条件是美国必须就其高超声速项目、全球反导系统以及在太空部署武器的计划进行详谈。里亚布科夫指出，俄罗斯对美国的上述计划感到担忧。"
        }

### 结果

In [ ]:
"主体名称-主体类型": [{"head": "高超声速项目、全球反导系统、太空部署武器", "tail": "高超声速项目、反导系统、太空武器"}

In [20]:
import json

input = {}
output = {}
# 打开JSONL文件
with open('newdata/test_demo.json', 'r', encoding='utf-8') as file:
    # 逐行读取文件
    for line in file:
        # 去除行尾的换行符并解析JSON
        data = json.loads(line.strip())
        # 处理解析后的字典
        
        print(data)

{'text': '在黑海水域中，俄罗斯黑海舰队导弹炮兵编队和堡垒机动式岸基反舰导弹系统小队进行了水面舰艇探测和打击的演习。在演习过程中，舞会和堡垒机动式岸基反舰导弹系统小队前往指定阵地，并对水面目标进行了识别和跟踪。根据指挥中心的指令，舞会和堡垒机动式岸基反舰导弹系统进入了战斗状态，并采用电子发射方式实施了对假想敌舰艇的导弹打击。', 'relation': [{'head': '黑海舰队导弹炮兵编队、堡垒机动式岸基反舰导弹系统小队', 'relation': '主体名称-主体类型', 'tail': '军事编队、军事小队'}, {'head': '黑海舰队导弹炮兵编队、堡垒机动式岸基反舰导弹系统小队', 'relation': '主体名称-主体数量', 'tail': '1、1'}]}
{'text': '瑞典国防军在哥特兰岛进行军事防御演练，指挥官艾米丽在两天的演习中指挥共15辆CV90步兵战车参与岛屿防御演习和快速部署行动，以加强岛屿的防御体系。', 'relation': [{'head': '岛屿防御演习', 'relation': '核心事件-时间', 'tail': '2024年8月6日'}, {'head': '岛屿防御演习', 'relation': '核心事件-组织机构', 'tail': '瑞典国防军'}, {'head': '岛屿防御演习', 'relation': '核心事件-地点', 'tail': '哥特兰岛'}, {'head': 'CV90步兵战车', 'relation': '主体名称-主体类型', 'tail': '装甲战斗车辆'}, {'head': 'CV90步兵战车', 'relation': '主体名称-主体数量', 'tail': '7'}]}
{'text': '预计今年年底，北德文斯克号将成为第一艘交付海军的亚森级潜艇，而第二艘喀山号则已于2009年开始建造，目前正在北方造船厂进行。', 'relation': [{'head': '交付', 'relation': '核心事件-时间', 'tail': '今年年底'}, {'head': '交付', 'relation': '核心事件-组织机构', 'tail': '海军'}, {'head': '亚森级潜艇', 'relation': '主体名称-主体类型',